In [137]:
import numpy as np
from numpy.linalg import norm

# Input

In [288]:
# Power function
def f(x):
    global counter
    counter += 1
    return((x[0]-1)**2 + (x[1]-1)**2)
#     return((1-x[0])**2 + 100*(x[1]-x[0]**2)**2)

# Rosenbrock function
# def f(x):
#     return((1-x[0])**2 + 100*(x[1]-x[0]**2)**2)

x0 = np.array([-1.2, 0])

# h = 10**-2
# h = 10**-3
h = 10**-5

eps = 10**-2

# Derivatives

In [289]:
def der_f_left(x, h, i):
    hi = np.array([0.0, 0.0])
    hi[i] = h
    return((f(x) - f(x-hi))/h)

def der_f_center(x, h, i):
    hi = np.array([0.0, 0.0])
    hi[i] = h
    return((f(x+hi) - f(x-hi))/(2*h))

def der_f_right(x, h, i):
    hi = np.array([0.0, 0.0])
    hi[i] = h
    return((f(x+hi) - f(x))/h)

def der_f(x, h, i):
    return(der_f_center(x, h, i))

def grad_f(x, h=h):
    return(np.array([
        der_f(x, h, i=0),
        der_f(x, h, i=1)
    ]))

In [290]:
def antigrad_direction(x):
    return(-grad_f(x, h) / norm(grad_f(x, h)))
antigrad_direction(x0)

array([0.91036648, 0.41380294])

# Swann

In [291]:
def swann(x, delta, direct):
    xs = [x]
    i = 0
    
    x_positive = xs[-1] + direct * delta * 2**i
    x_negative = xs[-1] - direct * delta * 2**i
#     if f(x) < f(x_negative) and f(x) < f(x_positive):
#         return([x, x])
    if f(x_negative) < f(x_positive):
        direct = -direct

    while True:
        x = xs[-1] + direct * delta * 2**i
        if f(xs[-1]) < f(x):
            x_half = xs[-1] + direct * delta * 2**(i-1)
            xs.append(x_half)
            xs.append(x)
            break;
        else:
            xs.append(x)
            i += 1

    if len(xs) > 3 and f(xs[-3]) < f(xs[-2]):
        interval = [xs[-4], xs[-2]]
    else:
        interval = [xs[-3], xs[-1]]
    return(interval)

interval = swann(x0, delta=h, direct=antigrad_direction(x0))
interval

[array([-0.00677355,  0.54237566]), array([2.37969754, 1.62713525])]

# Golden ratio

In [292]:
def golden_ratio(interval, direct):
    x_positive = interval[0] + direct
    x_negative = interval[1] - direct
    if f(x_negative) <= f(x_positive):
        direct = -direct

    step = 0.618
    a = interval[0]
    b = interval[1]
    l = b - a
    x1 = a + (1 - step) * norm(l) * direct
    x2 = a + step * norm(l) * direct

    while True:
        if f(x1) <= f(x2):
            a = a
            b = x2
            l = b - a
            x2 = x1
            x1 = a + (1 - step) * norm(l) * direct
        else:
            a = x1
            b = b
            l = b - a
            x1 = x2
            x2 = a + step * norm(l) * direct
        if norm(l) < eps:
            x = x1 if f(x1) < f(x2) else x2
            return(x)

x = golden_ratio(interval, direct=antigrad_direction(x0))
x

array([0.99891596, 0.99950725])

# DSK-Powell method

In [293]:
def dsk_powell(interval, direct):
    # DSK
    x1, x3 = interval
    x2 = (x1 + x3)/2
    delta_x = norm(x1-x2)
    x_star = x2 + delta_x * (f(x1) - f(x3)) / (2 * (f(x1) - 2*f(x2) + f(x3)))
    m = min([x1, x2, x3], key=lambda x: f(x))

    exit_criteria = norm(f(x1) - f(m)) < eps and norm(x1 - m) < eps

    xs = sorted([x1, x2, x3, x_star], key=lambda x: x[0])
    m = min(xs, key=lambda x: f(x))

    if f(xs[1]) == f(m):
        x1 = xs[0]
        x2 = xs[1]
        x3 = xs[2]
    else:
        x1 = xs[1]
        x2 = xs[2]
        x3 = xs[3]

        
    # Powell    
    while not exit_criteria:
        a1 = (f(x2) - f(x1)) / (x2 - x1)
        a2 = ((f(x3) - f(x1) / (x3 - x1)) - ((f(x2) - f(x1)) / (x2 - x1))) / (x3 - x2)
        x_star = ((x2 + x1) / 2) - a1 / (2 * a2)

        m = min([x1, x2, x3], key=lambda x: f(x))
        
        exit_criteria = norm(f(x1) - f(m)) < eps and norm(x1 - m) < eps

        xs = sorted([x1, x2, x3, x_star], key=lambda x: x[0])
        m = min(xs, key=lambda x: f(x))
        
        if f(xs[1]) == f(m):
            if f(x1) == f(xs[0]) and f(x2) == f(xs[1]) and f(x3) == f(xs[2]):
                return(m)
            x1 = xs[0]
            x2 = xs[1]
            x3 = xs[2]
        else:
            if f(x1) == f(xs[1]) and f(x2) == f(xs[2]) and f(x3) == f(xs[3]):
                return(m)
            x1 = xs[1]
            x2 = xs[2]
            x3 = xs[3]
    return(m)

x = dsk_powell(interval, direct=antigrad_direction(x0))
x

array([1.01032017, 0.97941742])

In [314]:
def find_min_in_direction(x0, direction):
    interval = swann(x0, delta=h, direct=direction)
#     x = dsk_powell(interval, direct=direction)
    x = golden_ratio(interval, direct=direction)
    return(x)

# Partan method

In [335]:
xs = [x0]
directions = []
grad_exit_criteria = False
eps = 10**-5
# h = 10**-3
counter = 0

while(not grad_exit_criteria):
    # antigrad direction
    direction = antigrad_direction(xs[-1])
    directions.append(direction)
    x = find_min_in_direction(xs[-1], directions[-1])
    xs.append(x)

    # antigrad direction
    direction = antigrad_direction(xs[-1])
    x = find_min_in_direction(xs[-1], direction)
    xs.append(x)

    # partan direction
    direction = xs[-3] - xs[-1]
    x = find_min_in_direction(xs[-1], direction)
    xs.append(x)

    grad_exit_criteria = norm(grad_f(xs[-1])) < eps
print(counter)
for i, x in enumerate(xs):
    print(f"f(x{i}) = f({x}) = {f(x)}")

368
f(x0) = f([-1.2  0. ]) = 5.840000000000001
f(x1) = f([-0.26869772  0.42331922]) = 1.9421546300571715
f(x2) = f([0.66260375 0.84663807]) = 0.13735611345192358
f(x3) = f([0.71281789 0.86946268]) = 0.09951355924800227
f(x4) = f([1.00000023 1.0000001 ]) = 6.294185203408995e-14
f(x5) = f([1.00000306 1.00000139]) = 1.1277242811351177e-11
f(x6) = f([1.00000387 1.00000176]) = 1.8073184130110885e-11


# Modified Partan method

In [336]:
xs = [x0]
counter = 0
# eps = 10**-3
# First step of a regular Partan method
# antigrad_direction
direction = antigrad_direction(xs[-1])
x = find_min_in_direction(xs[-1], direction)
xs.append(x)
# antigrad_direction
direction = antigrad_direction(xs[-1])
x = find_min_in_direction(xs[-1], direction)
xs.append(x)
# partan direction
direction = xs[-3] - xs[-1]
x = find_min_in_direction(xs[-1], direction)
xs.append(x)

grad_exit_criteria = False

while(not grad_exit_criteria):
    # antigrad_direction
    direction = antigrad_direction(xs[-1])
    x = find_min_in_direction(xs[-1], direction)
    xs.append(x)

    # modified partan direction
    direction = xs[-1] - xs[-4]
    x = find_min_in_direction(xs[-1], direction)
    xs.append(x)

    grad_exit_criteria = norm(grad_f(xs[-1])) < eps

print(counter)
for i, x in enumerate(xs):
    print(f"f(x{i}) = f({x}) = {f(x)}")

482
f(x0) = f([-1.2  0. ]) = 5.840000000000001
f(x1) = f([-0.26869772  0.42331922]) = 1.9421546300571715
f(x2) = f([0.66260375 0.84663807]) = 0.13735611345192358
f(x3) = f([0.71281789 0.86946268]) = 0.09951355924800227
f(x4) = f([1.00000023 1.0000001 ]) = 6.294185203408995e-14
f(x5) = f([1.00000459 1.00000208]) = 2.537211087138011e-11
f(x6) = f([1.00000741 1.00000337]) = 6.633047150545015e-11
f(x7) = f([1.00000823 1.00000374]) = 8.167645082964733e-11
f(x8) = f([1.00001106 1.00000503]) = 1.4749570427120327e-10
f(x9) = f([1.00000444 1.00000202]) = 2.3751941614296394e-11
